In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from scipy.io import wavfile
import librosa.display
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.models import load_model

import soundfile as sf
import librosa

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [2]:
path = 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/train'    #데이터폴더 경로 지정해주기 
parent_folders = os.listdir(path)                     #폴더 불러오기
path_li = []
total_sum = 0
for parent_folder in parent_folders:
    child_folders = []
    child_folders.append((np.array(os.listdir(path+"/"+parent_folder)).flatten().tolist())) # wav에 있는 폴더 다 불러오기
    temp_folders = (np.array(child_folders)).flatten().tolist()
    for folder in temp_folders:
        files = os.listdir(path+"/"+parent_folder+"/"+folder) #sessio01폴더 불러오기
        for file in files:
            if (path+"/"+parent_folder+'/'+folder+'/'+file).endswith('.wav') == True: # session01_impro01안에있는 txt 파일만 순서대로 path_li로 보내기
                path_li.append(path+"/"+parent_folder+"/"+folder+"/"+file)

path_li[:3]

['E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/train/Session01/Sess01_impro01/Sess01_impro01_F001.wav',
 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/train/Session01/Sess01_impro01/Sess01_impro01_F002.wav',
 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/train/Session01/Sess01_impro01/Sess01_impro01_F003.wav']

In [3]:
print(len(path_li))

9360


## 14-45초 음성 파일을 10초자리로 sliding_window 진행

In [4]:
# 14초~45초 되는 음성 파일들을 가져옴
audio_files = []
for filename in path_li: # 음성 파일이 저장된 폴더 경로
    y, sr = librosa.load(filename, sr=16000)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < 14 or duration >= 45:
        continue
    audio_files.append(filename)

In [ ]:
len(audio_files)

In [ ]:
# sliding window 크기 (10초)
window_size = 10

# sliding window 간격 (10초)
hop_size = 5

# 14초~45초 되는 음성 파일들을 저장할 폴더 경로
sliding_window = 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/train/sliding window'

In [ ]:
# sliding window을 적용하여 데이터 쪼개기
for file_path in audio_files:
    filename = os.path.basename(file_path)
    y, sr = librosa.load(file_path, sr=16000, offset=0)
    for i in range(0, len(y) - window_size * sr + hop_size * sr, hop_size * sr):
        if i + window_size * sr > len(y):
            window = y[-window_size * sr:]
        else:
            window = y[i:i + window_size * sr]
        output_file = os.path.join(sliding_window, f'{filename}_{i+window_size}s.wav')
        sf.write(output_file, window, sr, subtype='PCM_16')

In [ ]:
for file_delete in audio_files:  #14초이상 45초 미만 음성파일 삭제
    os.remove(file_delete)

## 음성 mfcc 추출

In [3]:
# 음성 파일을 feaher로 생성

max_pad_len = 3300
hop_length = 160  # 전체 frame 수 (hop_length의 길이만큼 옆으로 가면서 데이터를 읽는 옵션, 10ms를 기본으로 하고 있어 16000Hz인 음성에서는 160에 해당)
n_fft = 400       # frame 하나당 sample 수(일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있으며 16000Hz인 음성에서는 400에 해당하는 값) 

out = []
for file in path_li:
    data, sr = librosa.load(file)  # librosa로 음성 데이터 불러오기
    mfccs = librosa.feature.mfcc(y=data, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=128)  #mfcc로 음성을 array형태로 나타냄
    # print(mfccs)
    pad_width = max_pad_len - mfccs.shape[1]  # shape 만들기
    if pad_width > 0:
        mfcc_sh = np.pad(mfccs, pad_width=((0,0), (0, pad_width)), mode='constant')
        out.append(mfcc_sh)
    else:
        out.append(mfccs[:,:max_pad_len])
    # print(mfcc_sh.shape)    
out[0]

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\librosa\feature\spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


array([[-569.62384  , -543.4611   , -564.1664   , ...,    0.       ,
           0.       ,    0.       ],
       [  84.5566   ,   79.78537  ,   89.013435 , ...,    0.       ,
           0.       ,    0.       ],
       [   3.384592 ,  -11.527005 ,  -15.091857 , ...,    0.       ,
           0.       ,    0.       ],
       ...,
       [   1.2283611,    1.2559738,    3.4666843, ...,    0.       ,
           0.       ,    0.       ],
       [  -5.042471 ,   -5.5685244,   -4.6630964, ...,    0.       ,
           0.       ,    0.       ],
       [  -3.0495148,   -1.4369392,   -4.6570053, ...,    0.       ,
           0.       ,    0.       ]], dtype=float32)

In [5]:
len(out)

9360

In [6]:
seesid_li = []
label_li = []
for path in path_li:
    seesid = (path.split('/')[-1].split('.')[0])
    seesid_li.append(seesid)
seesid_df = pd.DataFrame(seesid_li)
seesid_df.columns = ['Segment ID']
seesid_df

,Segment ID
0,Sess01_impro01_F001
1,Sess01_impro01_F002
2,Sess01_impro01_F003
3,Sess01_impro01_F004
4,Sess01_impro01_F005
...,...
9355,Sess20_script05_M013
9356,Sess20_script05_M013
9357,Sess20_script06_M022
9358,Sess20_script06_M022


In [7]:
label_path = 'E:/Competition/ETRI Competition//Analysis/감정 레이블/train 감정_보이스'     # 감정레이블 경로설정
label_filename = os.listdir(label_path)                                                  # 경로 설정한 폴더 열기
label_li = [label_file for label_file in label_filename if label_file.endswith('.csv')]  # .csv 파일만 블러오기

In [8]:
# 감정라벨
total_label_df = pd.DataFrame()  
for filename in label_li:
    temp_label_df = pd.read_csv(label_path+'/'+filename)  # 감정레이블 파일 내용 읽어드리기
    total_label_df = pd.concat([total_label_df, temp_label_df], axis=0) 

total_label_df = total_label_df.reset_index(drop=True) 
print(len(total_label_df))

total_label_df[0:5]

8187


,Numb,Segment ID,Total Evaluation Emotion,Total Evaluation Valence,Total Evaluation Arousal,Eval01F Emotion,Eval01F Valence,Eval01F Arousal,Eval02F Emotion,Eval02F Valence,...,Eval15M Arousal,Eval16M Emotion,Eval16M Valence,Eval16M Arousal,Eval17M Emotion,Eval17M Valence,Eval17M Arousal,Eval19M Emotion,Eval19M Valence,Eval19M Arousal
0,1,Sess01_script01_M001,surprise,1.7,4.0,surprise,2.0,5.0,surprise,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Sess01_script01_F001,fear,1.5,3.6,fear,2.0,4.0,fear,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Sess01_script01_M002,angry,1.3,4.3,angry,1.0,5.0,angry,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Sess01_script01_M003,angry,1.5,3.8,angry,2.0,4.0,surprise,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Sess01_script01_F002,fear,1.6,3.7,fear,3.0,4.0,fear,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# wav 파일 기준으로 음성파일과 segment id가 동일하다면 Total Evaluation Emotion값을 가져옴
label_value_li = []
for i in range(len(seesid_df)):
   for k in range(len(total_label_df)):
           if seesid_df['Segment ID'][i] == total_label_df['Segment ID'][k]:
               label_value_li.append(total_label_df.loc[k, 'Total Evaluation Emotion'])

In [30]:
train_label=pd.DataFrame(label_value_li)
train_label
train_label.to_csv("final_train_label0854.csv", index = False)

In [4]:
train_label= pd.read_csv('E:/Competition/ETRI Competition/Analysis/final_train_label0854.csv')
train_label

,0
0,neutral
1,neutral
2,neutral
3,neutral
4,disgust
...,...
9355,neutral
9356,neutral
9357,angry
9358,angry


In [10]:
len(label_value_li)

9360

In [5]:
x_train = np.array(out)
x_train.shape

(9360, 128, 3300)

In [6]:
y = np.array(train_label)
y.shape

(9360, 1)

In [7]:
le = LabelEncoder()
y_train = to_categorical(le.fit_transform(y))
y_train.shape

c:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(9360, 7)

## CNN model 학습

In [11]:
n_columns = 3300    
n_row = 128       
n_channels = 1
n_classes = 7

# input shape 조정
x_train_input = tf.reshape(x_train, [-1, n_row, n_columns, n_channels])

In [12]:
model = keras.Sequential()

model.add(layers.Conv2D(input_shape=(n_row, n_columns, n_channels), filters=64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.1))
model.add(layers.Conv2D(kernel_size=2, filters=128, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.1))
model.add(layers.Conv2D(kernel_size=2, filters=128, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.1))
model.add(layers.Conv2D(kernel_size=2, filters=256, activation='relu'))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.1))

model.add(layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 3298, 64)     640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 1649, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 1649, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 1648, 128)     32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 824, 128)     0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 824, 128)      0

In [14]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)  # val_loss값이 10번정도 안떨어지면 학습 멈추게 하는 코드
mc = ModelCheckpoint('aaaaa.h5', monitor='loss', mode='min', save_best_only=True)

In [15]:
training_epochs = 15
num_batch_size = 64

learning_rate = 0.001
opt = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# 모델 학습
history = model.fit(x_train_input, y_train, batch_size=num_batch_size, epochs=training_epochs, callbacks=[es, mc])

Epoch 1/15
147/147 [==============================] - 2151s 15s/step - loss: 1.6370 - accuracy: 0.4021
Epoch 2/15
147/147 [==============================] - 2146s 15s/step - loss: 1.5097 - accuracy: 0.4524
Epoch 3/15
147/147 [==============================] - 2136s 15s/step - loss: 1.4912 - accuracy: 0.4591
Epoch 4/15
147/147 [==============================] - 2131s 14s/step - loss: 1.4739 - accuracy: 0.4667
Epoch 5/15
147/147 [==============================] - 2160s 15s/step - loss: 1.4695 - accuracy: 0.4705
Epoch 6/15
147/147 [==============================] - 2134s 15s/step - loss: 1.4570 - accuracy: 0.4730
Epoch 7/15
147/147 [==============================] - 2132s 15s/step - loss: 1.4402 - accuracy: 0.4778
Epoch 8/15
147/147 [==============================] - 2136s 15s/step - loss: 1.4266 - accuracy: 0.4857
Epoch 9/15
147/147 [==============================] - 2132s 15s/step - loss: 1.4117 - accuracy: 0.4933
Epoch 10/15
147/147 [==============================] - 2132s 15s/step - l

In [38]:
# 모델을 저장한 파일 경로 설정
model_path = 'E:/Competition/ETRI Competition/Analysis/final_model.h5'

In [41]:
# 모델 로드
audio_model = load_model(model_path)
audio_model

## test 예측

In [16]:
path_t = 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/test'    #데이터폴더 경로 지정해주기 
parent_folders_t = os.listdir(path_t)                     #폴더 불러오기
path_li_t = []
total_sum_t = 0
for parent_folder_t in parent_folders_t:
    child_folders_t = []
    child_folders_t.append((np.array(os.listdir(path_t+"/"+parent_folder_t)).flatten().tolist())) # wav에 있는 폴더 다 불러오기
    temp_folders_t = (np.array(child_folders_t)).flatten().tolist()
    for folder_t in temp_folders_t:
        files_t = os.listdir(path_t+"/"+parent_folder_t+"/"+folder_t) #sessio01폴더 불러오기
        for file_t in files_t:
            if (path_t+"/"+parent_folder_t+'/'+folder_t+'/'+file_t).endswith('.wav') == True: # session01_impro01안에있는 txt 파일만 순서대로 path_li로 보내기
                path_li_t.append(path_t+"/"+parent_folder_t+"/"+folder_t+"/"+file_t)

path_li_t[:3]

['E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/test/Session03/Sess03_impro01/Sess03_impro01_F001.wav',
 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/test/Session03/Sess03_impro01/Sess03_impro01_F002.wav',
 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/test/Session03/Sess03_impro01/Sess03_impro01_F003.wav']

In [17]:
len(path_li_t)

2095

### 14-45초 음성 파일을 10초자리로 데이터 shape 맞춰주기

In [61]:
# 14초이상 되는 음성 파일들을 가져옴
audio_files_t = []
for filename_t in path_li_t: # 음성 파일이 저장된 폴더 경로
    y_t, sr_t = librosa.load(filename_t, sr=16000)
    duration_t = librosa.get_duration(y=y_t, sr=sr_t)
    if duration_t < 14:
        continue
    audio_files_t.append(filename_t)

In [62]:
len(audio_files_t)  #14초이상 124

0

In [55]:
# 14초~45초 되는 음성 파일들을 저장할 폴더 경로
slid_test = 'E:/Competition/ETRI Competition/RAW DATA/KEMDy19/wav/test/slid_test'

In [56]:
segment_duration = 10

for file_path_t in audio_files_t:
    filename_t = os.path.basename(file_path_t)
    y_t, sr_t = librosa.load(file_path_t, sr=16000, offset=0)
    num_segments = int(np.ceil(len(y) / (segment_duration * sr)))
    for i in range(num_segments):
        start = i * segment_duration
        end = (i + 1) * segment_duration
        y_segment = y[int(start * sr):int(end * sr)]
        output_file_t = os.path.join(slid_test, f'{filename_t}.wav')
        sf.write(output_file_t, y_segment, sr_t, subtype='PCM_16')

In [57]:
for file_delete_t in audio_files_t:  #14초이상 45초 미만 음성파일 삭제
    os.remove(file_delete_t)

## test 예측

In [18]:
max_pad_len_t = 3300
hop_length_t = 160  # 전체 frame 수 (hop_length의 길이만큼 옆으로 가면서 데이터를 읽는 옵션, 10ms를 기본으로 하고 있어 16000Hz인 음성에서는 160에 해당)
n_fft_t = 400       # frame 하나당 sample 수(일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있으며 16000Hz인 음성에서는 400에 해당하는 값) 

out_t = []
for file_t in path_li_t:
    data_t, sr_t = librosa.load(file_t)  # librosa로 음성 데이터 불러오기
    mfccs_t = librosa.feature.mfcc(y=data_t, sr=sr_t, n_fft=n_fft_t, hop_length=hop_length_t, n_mfcc=128)  #mfcc로 음성을 array형태로 나타냄
    pad_width_t = max_pad_len_t - mfccs_t.shape[1]
    if pad_width_t < 0:
        mfcc_sh_t = mfccs_t[:, :max_pad_len_t]
    else:
        mfcc_sh_t = np.pad(mfccs_t, pad_width=((0, 0), (0, pad_width_t)), mode='constant')
    out_t.append(mfcc_sh_t)
out_t[0]

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\librosa\feature\spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


array([[-672.8785   , -671.7775   , -673.7934   , ...,    0.       ,
           0.       ,    0.       ],
       [  66.01541  ,   67.96032  ,   65.612915 , ...,    0.       ,
           0.       ,    0.       ],
       [  11.4209   ,   29.732376 ,   25.676243 , ...,    0.       ,
           0.       ,    0.       ],
       ...,
       [   1.1737795,    1.9922352,    1.7162523, ...,    0.       ,
           0.       ,    0.       ],
       [  -2.7105546,   -3.4214401,   -3.694456 , ...,    0.       ,
           0.       ,    0.       ],
       [  -0.7262459,   -3.2338257,   -3.091505 , ...,    0.       ,
           0.       ,    0.       ]], dtype=float32)

In [19]:
len(out_t)

2095

In [20]:
seesid_li_t = []
label_li_t = []
for path_t in path_li_t:
    seesid_t = (path_t.split('/')[-1].split('.')[0])
    seesid_li_t.append(seesid_t)
seesid_df_t = pd.DataFrame(seesid_li_t)
seesid_df_t.columns = ['Segment ID']
seesid_df_t

,Segment ID
0,Sess03_impro01_F001
1,Sess03_impro01_F002
2,Sess03_impro01_F003
3,Sess03_impro01_F004
4,Sess03_impro01_F006
...,...
2090,Sess17_script04_M012
2091,Sess17_script04_M014
2092,Sess17_script05_M023
2093,Sess17_script06_M013


In [21]:
label_path_t = 'E:/Competition/ETRI Competition/Analysis/감정 레이블/test 감정_보이스'  # 감정레이블 경로설정
label_filename_t = os.listdir(label_path_t)                                  # 경로 설정한 폴더 열기
label_li_t = [label_file_t for label_file_t in label_filename_t if label_file_t.endswith('.csv')]  # .csv 파일만 블러오기

In [22]:
# 감정라벨
total_label_df_t = pd.DataFrame()  
for filename_t in label_li_t:
    temp_label_df_t = pd.read_csv(label_path_t+'/'+filename_t)  # 감정레이블 파일 내용 읽어드리기
    total_label_df_t = pd.concat([total_label_df_t, temp_label_df_t], axis=0) 

total_label_df_t = total_label_df_t.reset_index(drop=True) 
print(len(total_label_df_t))

total_label_df_t[0:5]

2095


,Numb,Segment ID,Total Evaluation Emotion,Total Evaluation Valence,Total Evaluation Arousal,Eval01F Emotion,Eval01F Valence,Eval01F Arousal,Eval02F Emotion,Eval02F Valence,...,Eval17M Arousal,Eval18F Emotion,Eval18F Valence,Eval18F Arousal,Eval19M Emotion,Eval19M Valence,Eval19M Arousal,Eval20F Emotion,Eval20F Valence,Eval20F Arousal
0,1,Sess03_script01_M001,surprise,1.8,3.5,surprise,3.0,4.0,surprise,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Sess03_script01_F001,fear,1.7,3.7,fear,1.0,4.0,fear,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Sess03_script01_M002,angry,1.4,4.0,angry,1.0,5.0,angry,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Sess03_script01_M003,angry,1.5,3.9,angry,1.0,4.0,sad,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Sess03_script01_F002,fear,1.9,3.6,fear,1.0,4.0,surprise,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#wav와 label을 segment id기준으로 다른게 있는지 확인

label_value_li_t = []
non_li_t = []
non_seg_li_t = []
joungbok_li_t = []
for i_t in range(len(seesid_df_t)):
   check_t = 0
   for k_t in range(len(total_label_df_t)):
       if check_t == 0:
           if seesid_df_t['Segment ID'][i_t] == total_label_df_t['Segment ID'][k_t]:
               label_value_li_t.append(total_label_df_t['Total Evaluation Emotion'][k_t])
               check_t = 1
       elif check_t == 1:
           if seesid_df_t['Segment ID'][i_t] == total_label_df_t['Segment ID'][k_t]:
               joungbok_li_t.append(seesid_df_t['Segment ID'][i_t])
               check_t = 1
print(len(label_value_li_t))
print(len(joungbok_li_t))
print(joungbok_li_t)

2095
0
[]


In [69]:
# 두 데이터프레임에서 공통으로 포함된 Segment ID를 찾습니다.
common_ids = seesid_df_t['Segment ID'].isin(total_label_df_t['Segment ID'])

# 두 데이터프레임에서 공통으로 포함되지 않은 Segment ID를 찾습니다.
different_ids = ~common_ids

# different_ids를 사용하여 seesid_df_t에서 공통되지 않은 Segment ID를 찾습니다.
unique_seesid = seesid_df_t.loc[different_ids, 'Segment ID']

In [70]:
unique_seesid # 삭제

629    Sess04_impro03_F025
Name: Segment ID, dtype: object

In [24]:
x_test = np.array(out_t)
x_test.shape

(2095, 128, 3300)

In [25]:
y_test1 = np.array(label_value_li_t)
y_test1.shape

(2095,)

In [26]:
le1 = LabelEncoder()
arr_float = to_categorical(le1.fit_transform(y_test1))
y_test = arr_float.astype(np.int64)
print(y_test.dtype)
print(arr_float.shape)

int64
(2095, 7)


In [28]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)

## 예측모델 평가

In [42]:
predictions = audio_model.predict(x_test)

66/66 [==============================] - 106s 2s/step


In [51]:
real_label_li = []
pred_label_li = []
for i in range(len(y_test)):
    real_label_li.append(np.argmax(y_test[i]))
    pred_label_li.append(np.argmax(predictions[i]))

In [49]:
# 예측값을 데이터프레임으로 변환
#df = pd.DataFrame({'Prediction': pred_label_li})
#df.to_csv('voice_predictions.csv', index=False)

#print(df[:5])

,Prediction
0,4
1,5
2,4
3,4
4,4


In [52]:
counter = 0 
total = 0 
for k in range(len(real_label_li)):
    total += 1
    if real_label_li[k] == pred_label_li[k]:
        counter += 1
    else:
        pass

print("Accuracy: ", counter / total)

Accuracy:  0.4505966587112172


In [33]:
import tensorflow_addons as tfa

c:\Users\user\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [34]:
metric = tfa.metrics.F1Score(num_classes=7,average='macro', threshold=None, name='f1_score', dtype=None)

In [35]:
metric.update_state(y_test, predictions)
result = metric.result()
#result.numpy()
print("f1_score: ", result.numpy())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
f1_score:  0.29646128
